In [1]:
import os
import shutil
import hickle
import re

import pandas as pd
import tempfile
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import numpy as np
import PIL
from PIL import Image, ImageChops, ImageDraw
import torch
from torch import nn
import torchvision.models
import cv2

from pathlib import Path

import tensorflow as tf
from tensorflow import keras
import matplotlib.patches as patches

from tqdm import tqdm
from numba import jit, cuda
from utils import systemic_brightening
import warnings

In [2]:
# set directory
os.chdir("/users/riya/race/classifier_experiments") # which one? yep

# ignore warnings
warnings.filterwarnings("ignore")

# import model
segmentation_classifier = keras.models.load_model('models/MIMIC-256x25680-20-split-resnet-Float16_2-race_detection_rop_seg_data_rop_seg-0.001_20220321-054140_epoch:011.hdf5')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [3]:
img_arr = np.zeros((4546, 256, 256, 3))

In [4]:
img_arr[0].shape

(256, 256, 3)

In [5]:
def image_from_id(img_path, path_name):
    arr = np.array(Image.open(img_path + path_name))
    resized = cv2.resize(arr, (256,256))
    channels = np.repeat(resized[:, :, np.newaxis], 3, axis=2).reshape((256,256,3))
    
    return channels

In [6]:
def get_race_from_id(img_id, race_csv_path):

    
    race_data = pd.read_csv(race_csv_path)
    img_row = race_data.loc[race_data['image_id'] == int(img_id)] # they both must be ints
    img_row = img_row.reset_index(drop=True) # for .at to work
    img_race = img_row.at[0,'race']
    
    return img_race    

In [7]:
# run through classifier

@jit         
def predict_on_images(img_path, preds_df, colname,
                     skeleton, thresh_type, intensity_change, brighten_sum,
                     csv_name = "brightened_predictions", 
                     preds_path = "/users/riya/race/classifier_experiments/predictions/experiment1_plus_systemic_brightening/", 
                     race_csv_path = "/users/riya/race/csv/image_race_data.csv"): 
    
    img_files = os.listdir(dataset_path)
    num_images = len(img_files) - 4500
    
    id_arr = [0] * num_images
    race_arr = [0] * num_images
    img_arr = np.zeros((num_images, 256, 256, 3))
    
    preds_arr = [0] * num_images
    
    for i in tqdm(range(num_images)):
        channels = image_from_id(img_path, img_files[i])
        modified_img = systemic_brightening(channels, skeleton, thresh_type, intensity_change, brighten_sum,
                                           image_size = (256, 256))
        modified_img = np.array(modified_img) # .reshape((1,256,256,3)) np reshape, bc substitute? 
        img_arr[i] = modified_img
                       
        # getting id     
        img_id = re.findall(r'\d+', img_files[i])[0] # only one number
        id_arr[i] = int(img_id) # be sure it's int
        
        # print(type(id_arr[i]))
        
        # getting race
        img_race = get_race_from_id(img_id, race_csv_path)
        race_arr[i] = img_race
    
    preds_df['id'] = id_arr
    preds_df['race'] = race_arr
  
    # getting prediction    
    prediction = segmentation_classifier(img_arr)
    
    for i in range(num_images):
        preds_arr[i] = prediction.numpy()[i, 1] # returning the white prediction for each image
  
    preds_df[colname] = preds_arr           
    preds_df.to_csv(preds_path + csv_name + ".csv")

In [8]:
all_predictions = pd.DataFrame(columns = ['id', '30', '60', '90', '120', '150']) # from id I can get race

dataset_path = "/users/riya/race/dataset/segmentations/"

In [9]:
predict_on_images(dataset_path, all_predictions, '30', False, 'below', 'brighten', 30)

100%|██████████| 46/46 [00:07<00:00,  6.00it/s]
